<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Цель" data-toc-modified-id="Цель-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Цель</a></span></li><li><span><a href="#План-действий" data-toc-modified-id="План-действий-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>План действий</a></span></li></ul></li><li><span><a href="#Spacy" data-toc-modified-id="Spacy-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Spacy</a></span></li><li><span><a href="#TF-IDF" data-toc-modified-id="TF-IDF-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>TF-IDF</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Классификация текстов методами ML

### Цель

На основе корпуса с более 150тыс. документов построить модель ML для предсказания их токсичности.

### План действий

- очистка корпуса регулярными выражениями
- распознавание корпуса при помощи Spacy и его доочистка
- эмбеддинг Spacy и LGBM
- TF-IDF

In [1]:
#!pip3 install unidecode -U
#!pip3 install contractions -U
#!pip3 install spacy -U
#!pip3 install mapply -U

In [2]:
#преобразование в юникод
import unidecode
#работа с апострофами(don't -> do not)
import contractions
#многопоточный apply
import mapply
mapply.init(n_workers=-1)
import spacy
import re
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline

In [3]:
try:
    df = pd.read_csv('toxic_comments.csv')
except FileNotFoundError:
    df = pd.read_csv('datasets/toxic_comments.csv')
display(df.shape)
df.head(2)

(159571, 2)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0


In [4]:
#соотношение категорий в корпусе
_=df.toxic.value_counts(normalize=True).values
{'toxic_0':_[0],'toxic_1':_[1]}

{'toxic_0': 0.8983211235124177, 'toxic_1': 0.10167887648758234}

In [5]:
'''очистка регулярными выражениями
оставляем знаки препинания для Spacy, чтобы лучше понимала предложения.
'''

corpus = df.text
def clear_text(text):
    #преобразование к юникоду и нижнему регистру
    _ = unidecode.unidecode(text.lower()) 
    
    #замена спец.символов, веб-ссылок, хэштегов, на точки
    _ = re.sub(r'http\S+|www\S+|@\S+|#\S+|\n|\r|\t', '. ', _)
    
    #удаление всего, кроме основных символов
    _ = re.sub(r'[^a-z,.!\? \']',' ', _) 
    
    #работа с апострофами
    _ = contractions.fix(_) 
    _ = re.sub(r'\'',' ', _)
    
    #удаление пробелов перед точками
    _ = re.sub(r'\s+\.', '.', _)
    
    #ограничение на повторение символа (2 подряд)
    _ = re.sub(r'(.)\1{2,}', '\\1\\1', _ )
    
    #удаление лишних точек
    _ = re.sub(r'\.{2,}', '.', _)
    #_ = re.sub(r'\b\w\b', '', _)
    
    #удаление слов с длиной больше 20
    _ = re.sub(r'\b\w{20,}\b', '', _)
    _ = ' '.join([word for word in _.split()])
    return _
corpus = corpus.mapply(clear_text)

  0%|          | 0/24 [00:00<?, ?it/s]

In [6]:
corpus.values[0]

'explanation. why the edits made under my username hardcore metallica fan were reverted? they were not vandalisms, just closure on some gas after i voted at new york dolls fac. and please do not remove the template from the talk page since i am retired now.'

## Spacy

In [7]:
'''загрузка словаря spacy и пайплайн для работы с документами (исключим из него распознавание названий)
более 500тыс. векторов в словаре, длина каждого 300
'''

#!python -m spacy download en_core_web_lg
nlp = spacy.load(('en_core_web_lg'), disable=['ner', 'custom'])

In [8]:
'''проходимся генератором по корпусу, преобразуя каждый документ в чистый текст и в вектор
(среднее векторов слов). Если документ оказывается без вектора - заменяем его нулевым вектором длиной 300
'''
spacy_gen = nlp.pipe(corpus, n_process=-1, batch_size=100)
corpus_texts = []
corpus_vectors = []
for doc in tqdm(spacy_gen):
    doc_vector = np.mean([_.vector for _ in doc if (_.has_vector and not _.is_stop and not _.is_punct)], axis=0)
    if not doc_vector.shape:
        doc_vector = np.array([0]*300)
    doc_text = ' '.join(
        [_.lemma_ for _ in doc if (_.has_vector and not _.is_stop and not _.is_punct and len(_.lemma_)>2)])
    corpus_texts.append(doc_text)
    corpus_vectors.append(doc_vector)

166it [00:04, 127.40it/s]/home/rudbur/anaconda3/envs/praktikum/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/rudbur/anaconda3/envs/praktikum/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
159571it [12:48, 207.58it/s]


In [9]:
#делим корпус на обучающую и тестовую выборки, следя за соотношениями в них целевого признака
corpus_vectors = pd.DataFrame(corpus_vectors)
corpus_texts = pd.Series(corpus_texts)
X_train, X_test, y_train, y_test, X_texts_train, X_texts_test = train_test_split(
    corpus_vectors, df.toxic, corpus_texts, test_size=.25, stratify=df.toxic, random_state=42)

del corpus_vectors, corpus_texts, df, nlp, spacy_gen, corpus

In [10]:
%%time

param_distributions = {
    'n_estimators':[500],
    'learning_rate':[.1],
    'num_leaves':[60]
}

spacy_lgbm = LGBMClassifier(
    objective='binary', class_weight='balanced',
    boosting_type='goss', random_state=42, force_col_wise=True
)

search = RandomizedSearchCV(spacy_lgbm, scoring='f1', n_jobs=-1, refit=True, verbose=2, random_state=42, n_iter=1,
                           cv=3, param_distributions=param_distributions)
search.fit(X_train, y_train)

display(f'CV_f1_score: {search.best_score_}')
display(f'test_f1_score: {f1_score(y_test, search.best_estimator_.predict(X_test))}')

del spacy_lgbm

Fitting 3 folds for each of 1 candidates, totalling 3 fits


'CV_f1_score: 0.738376896109436'

'test_f1_score: 0.7409274193548387'

CPU times: user 4min 29s, sys: 3.18 s, total: 4min 32s
Wall time: 4min 3s


## TF-IDF

In [11]:
#пайплайн из TFIDF и линейной регрессии с подбором гиперпараметров (лучшие захардкодены)
tfidf = TfidfVectorizer(lowercase=False)
logreg = LogisticRegression(class_weight='balanced', n_jobs=-1, max_iter=500)
pipe = Pipeline([('tfidf', tfidf), ('logreg', logreg)])
param_distributions = {
    'tfidf__ngram_range':[(1,2)],
    'tfidf__max_features':[200000],
    'tfidf__max_df':[.4],
    'logreg__C':[6]
}

search = RandomizedSearchCV(pipe, scoring='f1', n_jobs=-1, refit=True, verbose=2, random_state=42, n_iter=25,
                           cv=3, param_distributions=param_distributions)

In [12]:
search.fit(X_texts_train,y_train)
f1_score(y_test, search.best_estimator_.predict(X_texts_test))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/home/rudbur/anaconda3/envs/praktikum/lib/python3.7/site-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 1 is smaller than n_iter=25. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


0.7763801537386443

## Вывод

Наиболее быстрой и точной оказалась модель TF-IDF + Регрессия, `f1 = 0.77`

- корпус был очищен от символов не из юникода, ссылок, хэштегов, повторяющихся символов методом регулярных выражения
- далее корпус был подан на вход Spacy. Распознав его, оставили в корпусе только леммы слов, для которых в словаре Spacy есть вектор, и которые не являются стоп-словами, знаками препинания и длиной больше 2 символов
- Spacy-вектора документов (300 признаков) отправили в LGBM, получив `f1 = 0.74`
- затем применили другой метод, проанализировав очищенный корпус методом TF-IDF и подобрав оптимальные гиперпараметры
- Логистическая регресссия дала `f1 = 0.77`